# Facial Expression Recognition Challenge

https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

From Kaggle:

The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

The training set consists of 28,709 examples. The public test set used for the leaderboard consists of 3,589 examples. The final test set, which was used to determine the winner of the competition, consists of another 3,589 examples.

Since Kaggle provides us with two test sets, we make the public test set our validation set and the private test set our test set. 

This does provide us with more training data as we don't have to partition a validation set out of our training set, so it does give us a slight advantage over the leaderboard. 

In [7]:
import numpy as np
import os
import pandas as pd
from imageio import imwrite

TRAIN_PATH = "data/train"
VALID_PATH = "data/valid"
TEST_PATH = "data/test"
DATASET_PATHS = (TRAIN_PATH, VALID_PATH, TEST_PATH)

OUTPUTS = 7
IMG_SIZE = 48

### Convert csv file to images and save to disk

In [2]:
df = pd.read_csv("data/fer2013.csv")

In [ ]:
# Create directory for train, valid, and test
for dataset_path in DATASET_PATHS:
    if not os.path.exists(dataset_path):
        os.mkdir(dataset_path)

    # Make a directory for each emotion
    for target in df["emotion"].unique():
        path_target = os.path.join(dataset_path, str(target))
        if not os.path.exists(path_target):
            os.mkdir(os.path.join(path_target))

for i, row in df.iterrows():
    # Convert image to numpy array and reshape to be 2d
    img = np.array(list(map(int, row["pixels"].split(" "))))
    img = img.reshape((IMG_SIZE, IMG_SIZE)).astype(np.uint8)

    # File name is <number>.png
    fname = str(i)+".png"

    if row["Usage"] == "Training":
        path = os.path.join(TRAIN_PATH, str(row["emotion"]))
    elif row["Usage"] == "PublicTest":
        path = os.path.join(VALID_PATH, str(row["emotion"]))
    elif row["Usage"] == "PrivateTest":
        path = os.path.join(TEST_PATH, str(row["emotion"]))
    else:
        raise Exception("Invalid Usage: {}".format(row["Usage"]))
        
    imwrite(os.path.join(path, fname), img)

### Build functions for transfer learning

In [4]:
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam

def transfer_CNN(base, outputs, lr=0.0005, dropout=0.5, decay=0.001):
    """ Appends a series of dense layers to any pretrained network and compiles for transfer learning.
    """
    model = Sequential([
        base,
        BatchNormalization(),
        GlobalAveragePooling2D(input_shape=base.output_shape[1:]),
        Dropout(dropout),
        Dense(1024, activation="relu"),
        BatchNormalization(),
        Dropout(dropout),
        Dense(1024, activation="relu"),
        BatchNormalization(),
        Dropout(dropout),
        Dense(outputs, activation="softmax"),
    ])
    opt = Adam(lr=lr, decay=decay)
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for data augmentation
# Rescale image to be between 0 and 1
# Allow horizontal flipping, x&y shifting, and zooming
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.10)

# Rescale validation and test images to be between 0 and 1
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

IN_IMG_SIZE = 200
BATCH_SIZE = 32
EPOCHS = 50

train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(IN_IMG_SIZE, IN_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    "data/valid",
    target_size=(IN_IMG_SIZE, IN_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)


def train(cnn, model_name, epochs=EPOCHS, batch_size=BATCH_SIZE):
    print("Training {}".format(model_name))
    
    callbacks = [
        TensorBoard(), 
        ModelCheckpoint("data/{}.h5".format(model_name), monitor="val_acc", save_best_only=True, save_weights_only=True),
        EarlyStopping(monitor="val_acc", patience=10)
    ]
    
    cnn.model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size + 1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // batch_size + 1,
        callbacks=callbacks)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


### Perform transfer learning on 4 state-of-the-art models from Keras applications

In [14]:
from keras.applications import InceptionResNetV2

base = InceptionResNetV2(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS, lr=0.0001)
train(model, "InceptionResNetV2")

Training InceptionResNetV2
Epoch 1/50
898/898 [==============================] - 461s 513ms/step - loss: 2.3416 - acc: 0.3059 - val_loss: 1.4345 - val_acc: 0.4723
Epoch 2/50
898/898 [==============================] - 451s 503ms/step - loss: 1.7153 - acc: 0.4280 - val_loss: 1.2717 - val_acc: 0.5132
Epoch 3/50
898/898 [==============================] - 453s 504ms/step - loss: 1.5050 - acc: 0.4759 - val_loss: 1.2260 - val_acc: 0.5534
Epoch 4/50
898/898 [==============================] - 453s 504ms/step - loss: 1.3678 - acc: 0.5126 - val_loss: 1.1621 - val_acc: 0.5634
Epoch 5/50
898/898 [==============================] - 452s 503ms/step - loss: 1.2660 - acc: 0.5445 - val_loss: 1.1112 - val_acc: 0.5876
Epoch 6/50
898/898 [==============================] - 452s 503ms/step - loss: 1.1936 - acc: 0.5672 - val_loss: 1.1038 - val_acc: 0.6004
Epoch 7/50
898/898 [==============================] - 452s 503ms/step - loss: 1.1431 - acc: 0.5852 - val_loss: 1.0817 - val_acc: 0.5993
Epoch 8/50
898/898 [=

In [15]:
from keras.applications import VGG16

base = VGG16(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "VGG")

Training VGG
Epoch 1/50
898/898 [==============================] - 398s 444ms/step - loss: 2.4281 - acc: 0.1914 - val_loss: 1.8139 - val_acc: 0.2402
Epoch 2/50
898/898 [==============================] - 392s 437ms/step - loss: 2.0583 - acc: 0.2108 - val_loss: 4.5340 - val_acc: 0.1184
Epoch 3/50
898/898 [==============================] - 393s 438ms/step - loss: 1.9482 - acc: 0.2278 - val_loss: 5.4302 - val_acc: 0.1819
Epoch 4/50
898/898 [==============================] - 393s 438ms/step - loss: 1.8598 - acc: 0.2508 - val_loss: 3.2674 - val_acc: 0.1819
Epoch 5/50
898/898 [==============================] - 393s 438ms/step - loss: 1.8316 - acc: 0.2557 - val_loss: 1.7283 - val_acc: 0.3112
Epoch 6/50
898/898 [==============================] - 393s 438ms/step - loss: 1.7565 - acc: 0.2820 - val_loss: 1.6796 - val_acc: 0.3040
Epoch 7/50
898/898 [==============================] - 393s 438ms/step - loss: 1.7191 - acc: 0.2984 - val_loss: 1.7211 - val_acc: 0.2705
Epoch 8/50
898/898 [===============

In [5]:
from keras.applications import ResNet50

base = ResNet50(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "ResNet")

Training ResNet
Epoch 1/50
898/898 [==============================] - 355s 396ms/step - loss: 1.8232 - acc: 0.4056 - val_loss: 1.2349 - val_acc: 0.5255
Epoch 2/50
898/898 [==============================] - 352s 392ms/step - loss: 1.2821 - acc: 0.5284 - val_loss: 1.1215 - val_acc: 0.5748
Epoch 3/50
898/898 [==============================] - 353s 393ms/step - loss: 1.1407 - acc: 0.5796 - val_loss: 1.0575 - val_acc: 0.6007
Epoch 4/50
898/898 [==============================] - 346s 385ms/step - loss: 1.0843 - acc: 0.5994 - val_loss: 1.0672 - val_acc: 0.5999
Epoch 5/50
898/898 [==============================] - 344s 383ms/step - loss: 1.0377 - acc: 0.6187 - val_loss: 1.0703 - val_acc: 0.5988
Epoch 6/50
898/898 [==============================] - 344s 383ms/step - loss: 0.9711 - acc: 0.6446 - val_loss: 0.9613 - val_acc: 0.6484
Epoch 7/50
898/898 [==============================] - 344s 384ms/step - loss: 0.9463 - acc: 0.6571 - val_loss: 0.9467 - val_acc: 0.6464
Epoch 8/50
898/898 [============

In [5]:
from keras.applications import InceptionV3

base = InceptionV3(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "Inception")

Training Inception
Epoch 1/50
898/898 [==============================] - 236s 263ms/step - loss: 2.1634 - acc: 0.3112 - val_loss: 1.4171 - val_acc: 0.4163
Epoch 2/50
898/898 [==============================] - 244s 272ms/step - loss: 1.6012 - acc: 0.4339 - val_loss: 1.2991 - val_acc: 0.5177
Epoch 3/50
898/898 [==============================] - 245s 273ms/step - loss: 1.3319 - acc: 0.5127 - val_loss: 1.1104 - val_acc: 0.5851
Epoch 4/50
898/898 [==============================] - 233s 259ms/step - loss: 1.2166 - acc: 0.5590 - val_loss: 1.1616 - val_acc: 0.5748
Epoch 5/50
898/898 [==============================] - 233s 259ms/step - loss: 1.1250 - acc: 0.5891 - val_loss: 1.0280 - val_acc: 0.6158
Epoch 6/50
898/898 [==============================] - 235s 261ms/step - loss: 1.0842 - acc: 0.6071 - val_loss: 0.9887 - val_acc: 0.6291
Epoch 7/50
898/898 [==============================] - 237s 264ms/step - loss: 1.0197 - acc: 0.6282 - val_loss: 0.9710 - val_acc: 0.6350
Epoch 8/50
898/898 [=========

### Build ensemble and check results

In [8]:
from keras.applications import InceptionResNetV2
from keras.applications import InceptionV3
from keras.applications import ResNet50
from keras.applications import VGG16

base = InceptionResNetV2(include_top=False, weights="imagenet")
inception_resnet = transfer_CNN(base, OUTPUTS)
inception_resnet.load_weights("data/InceptionResNetV2.h5")

base = VGG16(include_top=False, weights="imagenet")
vgg = transfer_CNN(base, OUTPUTS)
vgg.load_weights("data/VGG.h5")

base = ResNet50(include_top=False, weights="imagenet")
resnet = transfer_CNN(base, OUTPUTS)
resnet.load_weights("data/ResNet.h5")

base = InceptionV3(include_top=False, weights="imagenet")
inception = transfer_CNN(base, OUTPUTS)
inception.load_weights("data/Inception.h5")

models = [inception_resnet, vgg, resnet, inception]
# Weights for ensemble averaging
weights = [0.25, 0.25, 0.25, 0.25]

In [11]:
IN_IMG_SIZE = 200
BATCH_SIZE = 32


def predict_ensemble(models, imgs, weights):
    """ Average the predictions of all models to get ensemble prediction.
    """
    batch_size = imgs.shape[0]
    # Create empty array for model predictions
    predictions = np.zeros((len(models), batch_size, OUTPUTS))
    # Predict the class of the image for each model
    for i in range(len(models)):
        predictions[i] = models[i].predict(imgs)
    # Return the weighted average prediction of all models
    return np.argmax(np.average(predictions, axis=0, weights=weights), axis=-1).squeeze()


def test_models(models, weights, generator, batch_size=BATCH_SIZE):
    """ Print accuracy of models given generator
    """
    predictions = np.zeros(generator.n)
    targets = np.zeros(generator.n)

    for i, (imgs, target) in enumerate(generator):
        if i >= generator.n // batch_size + 1:
            break
        predictions[i*batch_size:(i+1)*batch_size] = predict_ensemble(models, imgs, weights)
        targets[i*batch_size:(i+1)*batch_size] = np.argmax(target, axis=1)
        
    accuracy = np.mean(targets == predictions)
    print("Accuracy: {}".format(round(accuracy, 5)))

In [12]:
valid_generator = test_datagen.flow_from_directory(
    "data/valid",
    target_size=(IN_IMG_SIZE, IN_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)

test_models(models, weights, valid_generator)

Found 3589 images belonging to 7 classes.
Accuracy: 0.7275


In [13]:
valid_generator = test_datagen.flow_from_directory(
    "data/test",
    target_size=(IN_IMG_SIZE, IN_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)

test_models(models, weights, valid_generator)

Found 3589 images belonging to 7 classes.
Accuracy: 0.73781


We achieve 72.75% accuracy on the public test set and 73.781% accuracy on the private test set.

This exceeds the top score of 69.768% on the public test and 71.161% on the private test set.